In [1]:
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup


# 将中文转换为unicode
def chinese2urlcode(str):
    uni = str.encode('unicode-escape').decode()
    return uni.replace('\\', '%')

# 获取网页
def askurl(url):
    headers = {
        "user-agent": "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.198 Safari/537.36 QIHU 360SE"
    }
    html = requests.get(url, headers=headers)
    return html.text

# 分割字符串：10.12-18.53万：[10.12, 18.53]
def split_str(str):
    site_1 = str.find('-')
    str1 = str[:site_1]
    site_2 = len(str)
    str2 = str[site_1+1:site_2-1]
    return str1, str2

def split_single(str):
    site1 = str.find('-')
    site2 = len(str) - 5
    return str[site1+1:site2-1]

# 把字符串转换为数字：18.53万->185600
def str2num(str):
    num = float(str) * 10000
    return num

def getcars():
    all_brands = []
    jili_brands = pd.read_csv('吉利.csv')
    for idx, row in jili_brands.iterrows():
        html = askurl(row['竞争品牌URL'])
        soup = BeautifulSoup(html, 'html.parser')
        div = soup.find('div', class_='content')
        h4 = div.find_all('h4', class_='rank-txt')
        dd = div.find_all('dd', class_='rcont-list')
        for i in range(len(h4)):
            other_brand = h4[i].find('a').text
            low_price, high_price = split_str(dd[i].find('span', class_='font-arial cd6-wei').text)
            if other_brand[:4] == '吉利汽车': continue
            all_brands.append([row['吉利车型'], row['低配价格'], row['高配价格'], other_brand[:other_brand.find('-')],split_single(other_brand), low_price, high_price])
    all_brands = np.array(all_brands)
    all_brands = pd.DataFrame({'吉利车型': all_brands[:, 0], '低配价格': all_brands[:, 1], '高配价格': all_brands[:, 2], '其它品牌': all_brands[:, 3],'其它车型': all_brands[:, 4], '其它车型低配价格': all_brands[:, 5], '其它车型高配价格': all_brands[:, 6]})
    all_brands.to_csv("竞品3.csv")
    # print(all_brands)


# getcars()
chinese2urlcode("星越L")

'%u661f%u8d8aL'